<a href="https://colab.research.google.com/github/drshahizan/special-topic-data-engineering/blob/main/assignment/data-scraping/submission/part2%20/MichelinStar/Web_Scraping_GS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen

In [47]:
import requests
from bs4 import BeautifulSoup

# Proxies api-endpoint 
URL = "http://api.proxiesapi.com"
  
# insert your auth key here
auth_key = "acd441ce6912262d127ed96b19875c26_sr98766_ooPq87"

no_pages = 40

alls = []
def get_data(pageNo):  

  url = "https://scholar.google.com/scholar?start='+str(pageNo)+'&q=Faculty+of+Computing%2C+Universiti+Teknologi+Malaysia&hl=en&as_sdt=0,5"
  # defining a params dict for the parameters to be sent to the API 
  PARAMS = {'auth_key':auth_key, 'url':url} 
  
  # sending get request and saving the response as response object 
  r = requests.get(url = URL, params = PARAMS) 
  soup = BeautifulSoup(r.content, 'html.parser')

  for d in soup.findAll('div', attrs={'class':'gs_r gs_or gs_scl'}):
        name = d.find('h3', attrs={'class':'gs_rt'})
        author = d.find('div', attrs={'class':'gs_a'})
        details = d.find('div', attrs={'class':'gs_rs'})
        users_cited = d.find('div', attrs={'class':'gs_fl gs_or_mvi'})
        conference_name = d.find('div', attrs={'class':'gs_or_ggsm'})
        publication_type = d.find('span', attrs={'class':'gs_ctg2'})
        # Find the <a> tag containing the keyword "Cited by"
        cited_by = d.find('a', string=re.compile(r'Cited by'))
        url = d.find('h3', attrs={'class':'gs_rt'})

        all1=[]

        if name is not None:
            all1.append(name.text)
        else:
            all1.append("No Title")

        # Extracting the author column
        if author is not None:
          match = re.search(r'^(.*?)\s*-', author.text)
          if match:
              author_list = match.group(1).strip()
              all1.append(author_list)
          else:
              all1.append("Not listed")
        else:    
            all1.append('0')

        if author is not None:
          # Extracting the second column
          match = re.search(r'-\s+(.*?)(?:,\s+\d{4}\s*-|$)', author.text)
          if match:
              second = match.group(1).strip()
              all1.append(second)
          else:
              all1.append("Not listed")
        else:    
            all1.append('0')

        if author is not None:
          # Extracting the publication date column
          match = re.search(r',\s+(\d{4})\s*-', author.text)
          if match:
              date = match.group(1).strip()
              all1.append(date)
          else:
              all1.append("Not listed")
        else:    
            all1.append('-')
        
        # Extracting the details column
        if details is not None:
            all1.append(details.text)
        else:
            all1.append('-')  

        # Extracting the conference_name column
        if conference_name is not None:
            match = re.search(r'\]\s*(.+)', conference_name.text)
            if match:
                conference = match.group(1)
                all1.append(conference)
            else:
                all1.append("Not listed")
        else:
            all1.append('Not listed')

        # Extracting the publication_type column
        if publication_type is not None:
            match = re.search(r'\[(.+)\]', publication_type.text)
            if match:
                pub = match.group(1)
                all1.append(pub)
        else:
            all1.append('Not listed')

        # Extract the desired information using regex
        if cited_by is not None:
            match = re.search(r'(Cited by)\s+(\d+)', cited_by.string)
            if match:
                count = match.group(2)
                all1.append(count)
            else:
                all1.append("0")
        else:
            all1.append("0")

        # Extracting the url column
        a_tag = url.find("a", href=True)
        if a_tag is not None:
            href = a_tag.get("href")
            all1.append(href)
        else:
            all1.append("No link")

        alls.append(all1)    

  return alls


In [52]:
results = []
for i in range(0, no_pages+1, 10):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Title', 'Author', 'Journal', 'Publication Year', 'Details', 'Conference Name', 'Publication Type', 'User Cited', 'URL'])
df.to_csv('google_scholar.csv', index=False, encoding='utf-8')

In [53]:
df = pd.read_csv("google_scholar.csv")

df.head(10)

,Title,Author,Journal,Publication Year,Details,Conference Name,Publication Type,User Cited,URL
0,… on multiple intelligence of students from th...,"F Zulkifli, A Fadhullah, N Abu Bakar…",Insight Journal …,2018,"… Universiti Teknologi MARA (UiTM), Perak and ...",uitm.edu.my,PDF,2,https://ir.uitm.edu.my/id/eprint/22515/
1,Use of e-learning by university students in Ma...,WM Al,Ieee …,2018,… the bachelor’s degree in computer science fr...,ieee.org,PDF,192,https://ieeexplore.ieee.org/abstract/document/...
2,Timetable scheduling system using genetic algo...,"HK Mammi, LY Ying",International Journal of Innovative Computing,2021,… Abstract—Current timetable scheduling system...,utm.my,PDF,4,https://ijic.utm.my/index.php/ijic/article/vie...
3,The effectiveness of using e-learning in Malay...,WM Al,Mediterranean Journal of Social …,2015,… the action of acquiring knowledge through co...,mcser.org,PDF,115,https://www.mcser.org/journal/index.php/mjss/a...
4,A new algorithm for improving the low contrast...,"Z Al‐Ameen, G Sulong",Scanning,2015,Contrast is a distinctive visual attribute tha...,Not listed,Not listed,31,https://onlinelibrary.wiley.com/doi/abs/10.100...
5,[PDF][PDF] “Faculty of Computer Science and In...,"S Defit, MNM Sap",core.ac.uk,Not listed,"The widespread use of computer applications, d...",core.ac.uk,PDF,0,https://core.ac.uk/download/pdf/42910251.pdf
6,"Universiti Teknologi Malaysia, Malaysia",AR Roselainy,… Learning of Mathematics at University Level:...,2002,"… On the other hand, the department also teach...",Not listed,Not listed,1,https://link.springer.com/content/pdf/10.1007/...
7,The feasibility of human haptic emotion as a f...,"AH Basori, D Daman, A Bade, MS Sunar…",Proceedings of The 7th …,2008,… Ahmad Hoirul Basori Department of Computer G...,Not listed,Not listed,30,https://dl.acm.org/doi/abs/10.1145/1477862.147...
8,"[PDF][PDF] … , Department of Computing, Univer...",VK Bardsiri,International Journal of Enterprise Information …,2014,Although perceived organizational support (POS...,archive.org,PDF,0,https://scholar.archive.org/work/kijrroap5fc6j...
9,Cyber security awareness among university stud...,"A Garba, MB Sirat, S Hajar…",Science Proceedings …,2020,"… at the school of computing, faculty of engin...",readersinsight.net,PDF,22,https://readersinsight.net/SPS/article/view/1320
